In [ ]:
## https://www.quantopian.com/posts/the-social-media-trader-mood-series-introduction のお勉強

# Importing Data
from __future__ import division
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import pyfolio as pf
import matplotlib.pyplot as plt
import seaborn as sns
from quantopian.pipeline import Pipeline
from quantopian.pipeline import CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import Latest
from itertools import chain
from datetime import date
from blaze import by, merge
from odo import odo
from quantopian.pipeline.classifiers.morningstar import Sector

import alphalens

SECTOR_NAMES = {
 101: 'Basic Materials',
 102: 'Consumer Cyclical',
 103: 'Financial Services',
 104: 'Real Estate',
 205: 'Consumer Defensive',
 206: 'Healthcare',
 207: 'Utilities',
 308: 'Communication Services',
 309: 'Energy',
 310: 'Industrials',
 311: 'Technology' ,
}

# Plotting colors
c = "#38BB86"

# Fundamentals for sector data
fundamentals = init_fundamentals()

# Importing our Data/Sample Version 
# PsychSignal's Twitter & StockTwits with Retweets sample is available from 24 Aug 2009 - 09 Jan 2016
from quantopian.interactive.data.psychsignal import aggregated_twitter_withretweets_stocktwits_free as dataset
# from quantopian.interactive.data.psychsignal import aggregated_twitter_withretweets_stocktwits as dataset


import warnings
warnings.filterwarnings('ignore')

In [ ]:
len(dataset.symbol.distinct())

In [ ]:
dataset[:3]

- **asof_date**:データを追加した日(apply to: 充当する). この日付が，このデータの日付．
- **symbol**: 
- **source**: (全部同じ)
- **bull_scored_messages**: PSアルゴリズムがスコアリングしたブルメッセージの数
- **bear_scored_messages**: PSアルゴリズムがスコアリングしたベアメッセージの数
- **bullish_intensity**: 0-4でスコアリングされた，それぞれのMSGのbull度．4が一番ブルブル．4はほとんど出てこない
- **bearish_intensity**: 0-4でスコアリングされた，それぞれのMSGのbear度．4が一番ベアベア．4はほとんど出てこない
- **total_scanned_messages**: PSに流されてきたmsgの数．PSのアルゴに組み入れられたかどうかは関係なしにすべてのかず
- **timestamp**: PS側でデータを記録した時間
- **bull_minus_bear**: [BULL - BEAR] 度．
- **bull_bear_msg_ratio**: bull bear メッセージ比率
- **sid**: 


In [ ]:
aaple = dataset[dataset.symbol == "XIV"]
aaple

In [ ]:
# historically にアップルのブルメッセージの数を点で表現
df_aaple = odo(aaple.sort('asof_date'), pd.DataFrame)
plt.plot(df_aaple.asof_date, df_aaple.bull_scored_messages, marker='.', linestyle='None', color='r')

#30日の平均
plt.plot(df_aaple.asof_date, df_aaple.bull_scored_messages.rolling(window=30,center=False).mean())

ax = plt.twinx() 
price = get_pricing("XIV", start_date=df_aaple.asof_date.iloc[0],  end_date=df_aaple.asof_date.iloc[-1], fields='price')
ax.plot(price)

In [ ]:
df_aaple.asof_date.iloc[-1]

In [ ]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage, Returns, Latest
from quantopian.pipeline.filters import Q500US, StaticAssets
from quantopian.interactive.data.quandl import yahoo_index_vix
from quantopian.pipeline.filters import StaticAssets
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.classifiers.morningstar import Sector


# PIPELINE を使うとき
from quantopian.pipeline.data.psychsignal import stocktwits_free as st

from zipline.utils.tradingcalendar import trading_day  
from pandas import date_range 
import pandas as pd
from scipy import stats

import numpy as np
import matplotlib.pyplot as plt

class CloseOnN(CustomFactor):  
    # Define inputs
    inputs = [USEquityPricing.close]
    window_length = 2 
    
    def compute(self, today, assets, out, close):  
        out[:] = close[0]

class MassageMA(CustomFactor):
    inputs = [st.total_scanned_messages]
    window_length = 10
    
    def compute(self, today, assets, out, msgs):  
        out[:] = msgs.mean()
    



In [ ]:
def my_pipe(asset_list):
    pipe = Pipeline()
    sma_10 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length=10)
    sma_50 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length=50)

    close = Latest(inputs = [USEquityPricing.close])
    close_1 = CloseOnN(window_length = 1)
    close_2 = CloseOnN(window_length = 2)
    close_3 = CloseOnN(window_length = 3)
    close_4 = CloseOnN(window_length = 4)
    close_5 = CloseOnN(window_length = 5)

    pipe.add(st.bull_bear_msg_ratio .latest, 'bull_bear_msg_ratio')
    pipe.add(st.bull_scored_messages .latest, 'bull_scored_messages')
    pipe.add(st.bear_scored_messages .latest, 'bear_scored_messages')
    pipe.add(st.total_scanned_messages .latest, 'total_scanned_messages')
    pipe.add(st.bearish_intensity .latest, 'bearish_intensity')
    pipe.add(st.bullish_intensity .latest, 'bullish_intensity')

    pipe.add(sma_10, "sma_10")
    pipe.add(sma_50, "sma_50")
    pipe.add(close, "close")

    pipe.add(close_1, 'close_1')
    pipe.add(close_2, 'close_2')
    pipe.add(close_3, 'close_3')
    pipe.add(close_4, 'close_4')
    pipe.add(close_5, 'close_5')
    pipe.add(Sector(), "Sector")

#     myasset = StaticAssets(symbols(['AAPL', 'XLK']))
#     myasset = StaticAssets(symbols(asset_list))
    mymask = st.bull_bear_msg_ratio.latest > 1/0.35
    pipe.set_screen(
#         myasset
         Q500US()
#         & mymask
#         & (sma_10 > sma_50) 
    #     & (st.bear_scored_messages.latest > 10)
    #     & (st.bull_bear_msg_ratio.latest > 1.0 / 0.35)
    )
    return pipe

def run_my_pipe(pipe, start, end):
    print "pipeline outputting... "
    pipe_output = run_pipeline(pipe, start_date=start, end_date=end)
#     print "making dataframe for messages ,,,"
#     dates = pipe_output.index.get_level_values(0).unique()
#     dates = dates.tz_localize(None)
    return pipe_output

def to_panel(pipe_output):
    pan = pipe_output.to_panel()
    pan = pan.rename(minor_axis=dict([(x,x.symbol) for x in pan.minor_axis]))
    return pan 
    
def ln_price_change(pan, n):
    df_close = pan.close 
    df = df_close.divide(df_close.shift(n)).apply(np.log)
    return df

def asset_etf_msg_ratio(pan, asset, etf):
    df_msg = pan.total_scanned_messages
    s_msg_ratio = df_msg[asset].apply(np.log) - df_msg[etf].apply(np.log) 
    return s_msg_ratio

######### 

def make_data(asset_list):
    pipe = my_pipe(asset_list)
    pipe_output = run_my_pipe(pipe, "2015-1-1", "2017-8-18")
    pan = to_panel(pipe_output)
    print("make data done")
    return pan

def plot(pan, asset, etf, n ):
    print "plotting %s and %s" % (asset, etf)
    df_change = ln_price_change(pan, n)
    s_asset_etf_msg_ratio = asset_etf_msg_ratio(pan,  asset, etf)
    df_change["msg_ratio"] = s_asset_etf_msg_ratio   
    df_change["msg_ratio_shift"] = df_change["msg_ratio"].shift(n)
    ax = df_change.plot(kind='scatter', x="msg_ratio_shift", y=asset)
    ax.set_title( "plotting %s / %s" % (asset, etf))

######### 

# pan = make_data([
#     "AAPL", "MSFT", "FB", "XLK", "QQQ", "VGT","CSCO",'ORCL',"IBM", "AMZN", 
#                  "JPM", "WFC", "BAC", "XLF",
#                  "XLE","XOM","CVX", "SLB","COP","EOG","OXY",
#                  "XBI","KITE","EXEL","ALXN","VRTX","GILD","BBH","AMGN", "CELG", "AGN", "REGN"
                #"XLY", "CMCS", "HD","DIS","MCD","PCLN","SBUX","NFLX","NKE",
#     "XLV", "JNJ", "PFE", "UNH", "MRK", "AMGN", "ABBV",
#     "XLI", "GE", "BA", "MMM", "HON", "UNP", "UTX", "LMT", "UPS", "CAT","ITW",
#     "XLU", "NEE","DUK",
    
#                 ]) 

# plot(pan, "NEE", "XLU", 4 )
# plot(pan, "DUK", "XLU", 4 )


In [ ]:
pipe = my_pipe([])
results = run_pipeline(pipe, '2015-1-1', '2017-1-11')

In [ ]:
len(results[results["Sector"] == 206].index.get_level_values(1).unique())


In [ ]:
idx = pd.IndexSlice
clm = "bull_bear_msg_ratio"
sector = 206
targets = results[results["Sector"] == sector].index.get_level_values(1).unique()[:20]
num = len(targets)
print num
xs = 20
ys = 5 #int(num/20)+1
fig = plt.figure(figsize=(10,10))


for i, sym in enumerate(targets ):
    df = results.loc[idx[:, [sym]], :]
    msg_mean = df[clm].mean()
    df["zscore"] = stats.zscore(df[clm])
    df["SMA"] =df[clm].rolling(window=30,center=False).mean() 
    df["STD"] =  df[clm].rolling(window=30,center=False).std()
    df["Mzscore"] = (df[clm] - df["SMA"] ) / df["STD"]
    df["Return"] = df.close.pct_change()
    x = i % xs
    y = int(i/xs)+ys
    
    ax = fig.add_subplot(xs, ys, i+1)
    ax.set_title(sym.symbol)
    ax.scatter(
        df["Mzscore"],
        df["Return"],
        alpha=0.3)



In [ ]:
ticker = results.index.levels[1].unique()
pricing = get_pricing(ticker, start_date='2015-1-1', end_date='2017-9-1', fields='open_price')

In [ ]:
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(results['bull_bear_msg_ratio'],
                                                                   pricing, 
                                                                   quantiles = None, 
                                                                   bins=5,
                                                                   groupby = results['Sector'],
                                                                
                                                                  )


In [ ]:
# alphalens.tears.create_full_tear_sheet(factor_data)
factor_data.head()

In [ ]:
mean_return_by_q, std_err = alphalens.performance.mean_return_by_quantile(factor_data,  by_group=True)


In [ ]:
alphalens.plotting.plot_quantile_returns_bar(mean_return_by_q,  by_group=True)

In [ ]:
mean_return_by_q_daily, std_err = alphalens.performance.mean_return_by_quantile(factor_data, by_date=True)
alphalens.plotting.plot_quantile_returns_violin(mean_return_by_q_daily,)

In [ ]:
alpha_beta = alphalens.performance.factor_alpha_beta(factor_data)
alpha_beta